In [1]:
import csv
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
answer_blue = pd.read_csv('./answer_key.blue.txt', header = None)
answer_blue.set_axis(['Protein'], axis='columns', inplace = True)

answer_green = pd.read_csv('./answer_key.green.txt', header = None)
answer_green.set_axis(['Protein'], axis='columns', inplace = True)

In [2]:
iPRG_MW = pd.read_csv('./iPRG2009_Study.MW.csv')
iPRG_MW.set_index('proteinIds', inplace = True)
iPRG_MW = iPRG_MW.astype({'mw':'float'})

In [3]:
# Comet

Colors = ['red', 'yellow', 'green', 'blue']
CNumber = [5,5,3,3]
Mean_RY = df()
Mean_G = df()
Mean_B = df()

for order, color in enumerate(Colors):
    countdf = df()

    for i in range(1,CNumber[order]+1):
        data = []
        filename = "./iPRG2009_ECOLI_"+color+"_"+str(i)+".iPRG2019_Study.comet.psms_pout.txt"
        f = open(filename, "rt")
        print(filename)
        reader = csv.reader(f, delimiter='\t')
        data = list(reader)
        data_list = df(data[1:])
        data_list = data_list.iloc[:,:6]
        data_list.columns = data[0]
        f.close()

        data_list = data_list.astype({'q-value':'float', 'posterior_error_prob':'float', 'score':'float'}) 

        CNTM = []
        for p in data_list['proteinIds']:
            CNTM.append(p.find('Cntm_'))
        data_list['isCNTM'] = CNTM
        data_list.drop(data_list[data_list['isCNTM'] != -1].index, inplace = True)

        data_cut = data_list[data_list['q-value'] < 0.05]
        data_group = data_cut.groupby('proteinIds')

        cnt = df(data_group.count().iloc[:,0]).transpose()
        cnt.index = [str(i)]
        if i == 1 : countdf = cnt
        else: countdf = countdf.append(cnt)

    countdf = countdf.fillna(0)
    if order == 0: Mean_RY =  df(countdf.mean()).transpose()
    elif order == 1: Mean_RY = Mean_RY.append(df(countdf.mean()).transpose())
    elif order == 2: Mean_G = df(countdf.mean())
    elif order == 3: Mean_B = df(countdf.mean())
        
Mean_G.columns = ['green']
Mean_B.columns = ['blue']        

Mean_RY = Mean_RY.transpose().fillna(0)
Mean_RY.columns = ['red', 'yellow']

red_yel_Diff = df(Mean_RY['red']-Mean_RY['yellow'])
red_yel_Diff.columns = ['Difference']
red_yel_Diff_copy = red_yel_Diff.copy()

In [4]:
standard = 10
for ind, diff in enumerate(red_yel_Diff['Difference']):
    if Mean_RY['red'][ind] >= standard and Mean_RY['yellow'][ind] >= standard:
        red_yel_Diff.iloc[ind] = 0

In [5]:
blue_est = red_yel_Diff[red_yel_Diff['Difference'] > 2]
green_est = red_yel_Diff[red_yel_Diff['Difference'] < -2]

green_exp = Mean_G[Mean_G['green'] > 1]
blue_exp = Mean_B[Mean_B['blue'] > 1]